In [1]:
import optuna
import yaml
import time
import numpy as np
import pandas as pd
import torch
from multiprocessing import Pool

from ili.dataloaders import StaticNumpyLoader
from ili.inference import InferenceRunner
from ili.validation import ValidationRunner
import ili

from CASBI.utils.create_dataframe import rescale

device='cuda'

N_subhalos = 2
data = pd.read_parquet('../../../../data/dataframe/dataframe.parquet')
data = rescale(data, mean_and_std_path='../../../../data/preprocess/mean_and_std.parquet', scale_observations=True, scale_parameter=True, inverse=True) 
data =  data.drop(['gas_log10mass', 'a','redshift', 'mean_metallicity', 'std_metallicity','mean_FeMassFrac', 'std_FeMassFrac', 'mean_OMassFrac', 'std_OMassFrac'], axis=1)
min_feh, max_feh = min(data['feh']), max(data['feh'])
min_ofe, max_ofe = min(data['ofe']), max(data['ofe'])
conditions = data[data.columns.difference(['feh', 'ofe', 'Galaxy_name'], sort=False)].drop_duplicates()

minimum_theta = [conditions[col].values.min() for col in conditions.columns]   
maximum_theta = [conditions[col].values.max() for col in conditions.columns]       
minimum_theta = np.array(minimum_theta)
maximum_theta = np.array(maximum_theta)
def repeat_array(arr, repetitions):
    return np.repeat(arr, repetitions)
repeat_minimum_theta = repeat_array(minimum_theta, N_subhalos)
repeat_maximum_theta = repeat_array(maximum_theta, N_subhalos) 

prior = ili.utils.Uniform(low=repeat_minimum_theta, high=repeat_maximum_theta, device=device)

N_test = 1_000
def preprocess_testset(i):
    galaxies = set(data['Galaxy_name'].drop_duplicates().sample(N_subhalos, random_state=i))
    parameters =  data[data['Galaxy_name'].isin(galaxies)].drop(['feh', 'ofe', 'Galaxy_name'], axis=1).drop_duplicates().values.T.reshape(-1)
    galaxy_data = data[data['Galaxy_name'].isin(galaxies)].values
    histogram_galaxy, _, _ = np.histogram2d(galaxy_data[:, 0], galaxy_data[:, 1], bins=64, range=[[min_feh, max_feh], [min_ofe, max_ofe]])
    sim_data =  np.expand_dims(np.log10(histogram_galaxy + 1e-6 +1), axis=0)
    return parameters, sim_data, galaxies

# Create a pool of workers
with Pool() as pool:
    # Map the function to the data
    results = pool.map(preprocess_testset, range(N_test))
    
# Unpack the results
theta_test, x_test, galaxies_test = zip(*results)

#take the first test set element as x_0 and theta_0    
galaxies_0 = galaxies_test[0]
data_to_plot_halos = data[data['Galaxy_name'].isin(galaxies_0)].to_parquet('./halos_0.parquet')
theta_0 =  theta_test[0]
x_0 =  x_test[0]

N = 10_000
def process_sample(i):
    galaxies = data['Galaxy_name'].drop_duplicates().sample(N_subhalos, random_state=i+int(time.time()))
    while (any(set(galaxies) == galaxy_in_testset for galaxy_in_testset in galaxies_test)):
        print('matched galaxies, try again')
        print('galaxies', set(galaxies))
        print('test galaxies', galaxies_test)
        galaxies = data['Galaxy_name'].drop_duplicates().sample(N_subhalos, random_state=i)
    parameters =  data[data['Galaxy_name'].isin(galaxies)].drop(['feh', 'ofe', 'Galaxy_name'], axis=1).drop_duplicates().values.T.reshape(-1)
    galaxy_data = data[data['Galaxy_name'].isin(galaxies)].values
    histogram_galaxy, _, _ = np.histogram2d(galaxy_data[:, 0], galaxy_data[:, 1], bins=64, range=[[min_feh, max_feh], [min_ofe, max_ofe]])
    sim_data =  np.expand_dims(np.log10(histogram_galaxy + 1e-6 +1), axis=0)
    return parameters, sim_data

# Create a pool of workers
with Pool() as pool:
    # Map the function to the data
    results = pool.map(process_sample, range(N))

# Unpack the results
theta, x = zip(*results)

#save in .npy files, we remove the first element of the test set since it will be stored as x_0 and theta_0
np.save('./x_test.npy', x_test[1:])
np.save('./theta_test.npy', theta_test[1:])
np.save('./x_0.npy', x_0)
np.save('./theta_0.npy', theta_0)
np.save('./x.npy', x)
np.save('./theta.npy', theta)
print('finish prepare the data')


matched galaxies, try again
galaxies {'g4.86e10.00784', 'g7.66e11.00240'}
test galaxies ({'g2.37e10.00880', 'g6.96e10.00368'}, {'g3.23e11.00784', 'g2.39e11.00400'}, {'g7.05e09.00288', 'g1.59e11.00960'}, {'g6.91e10.00192', 'g3.54e09.00192'}, {'g1.47e10.00944', 'g3.67e10.00640'}, {'g8.28e11.00496', 'g1.89e10.00944'}, {'g2.64e10.00480', 'g8.06e11.00256'}, {'g4.94e10.00784', 'g6.31e09.00400'}, {'g1.23e10.01008', 'g3.59e11.00320'}, {'g6.12e10.00672', 'g1.47e10.00832'}, {'g4.94e10.00368', 'g6.31e09.00688'}, {'g3.19e10.01008', 'g3.59e11.00224'}, {'g3.54e09.00928', 'g2.19e11.00928'}, {'g2.54e11.00064', 'g1.59e11.00944'}, {'g4.48e10.00352', 'g6.96e10.00720'}, {'g5.05e10.00656', 'g2.39e10.00256'}, {'g5.46e11.00752', 'g1.08e11.00672'}, {'g1.18e10.01024', 'g2.42e11.00896'}, {'g2.39e10.00320', 'g6.31e09.00432'}, {'g4.94e10.00080', 'g4.48e09.00912'}, {'g1.88e10.00128', 'g1.89e10.00480'}, {'g2.37e10.00656', 'g3.06e11.00208'}, {'g6.96e10.00128', 'g4.90e11.00704'}, {'g9.91e09.00752', 'g6.31e09.00240'},

In [2]:
import torch.nn as nn

class ConvNet(nn.Module):
    def __init__(self, input_channel, output_dim):
        super(ConvNet, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(input_channel, 8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(32 * 8 * 8, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        out = self.conv_layers(x)
        out = out.view(out.size(0), -1)
        out = self.fc_layers(out)
        return out

In [3]:
def objective(trial):
    # Suggest values for the hyperparameters
    model = trial.suggest_categorical('model', ['maf', 'nsf'])
    hidden_features = trial.suggest_categorical('hidden_features', [10, 50, 70, 100])
    num_transforms = trial.suggest_categorical('num_transforms', [5, 10, 15, 20])
    learning_rate = trial.suggest_categorical('learning_rate', [1e-5, 5e-5, 1e-4]) #suggest_loguniform('learning_rate', 1e-5, 1e-4)
    output_dim = trial.suggest_categorical('output_dim', [5, 10, 32, 64])
        
    # reload all simulator examples as a dataloader
    all_loader = StaticNumpyLoader.from_config("./data.yaml")
    
    embedding_net = ConvNet(input_channel=1, output_dim=output_dim)
    
    nets = [ ili.utils.load_nde_sbi(engine='NPE', model=model, hidden_features=hidden_features, num_transforms=num_transforms, embedding_net=embedding_net)]

    train_args = {
    'training_batch_size': 1024,
    'learning_rate': learning_rate,
    'stop_after_epochs': 20}
    
    runner = InferenceRunner.load(
    backend='sbi',
    engine='NPE',
    prior=prior,
    nets=nets,
    device=device,
    embedding_net=embedding_net,
    train_args=train_args,
    proposal=None,
    out_dir=None,)

    # train a model to infer x -> theta. save it as toy/posterior.pkl
    # runner = InferenceRunner.from_config(f"./training.yaml")
    _, summaries = runner(loader=all_loader)
    
    return summaries[0]['validation_log_probs'][-1]

In [4]:
study =  optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, n_jobs=10)

[I 2024-05-29 14:15:12,459] A new study created in memory with name: no-name-2ed14fc1-7a33-45cf-8e72-450914ec2c15
INFO:root:MODEL INFERENCE CLASS: NPE
INFO:root:MODEL INFERENCE CLASS: NPE
/export/home/vgiusepp/.local/lib/python3.10/site-packages/sbi/utils/torchutils.py:27: UserWarning: GPU was selected as a device for training the neural network. Note that we expect **no** significant speed ups in training for the default architectures we provide. Using the GPU will be effective only for large neural networks with operations that are fast on the GPU, e.g., for a CNN or RNN `embedding_net`.
  warnings.warn(
INFO:root:MODEL INFERENCE CLASS: NPE
INFO:root:MODEL INFERENCE CLASS: NPE
INFO:root:MODEL INFERENCE CLASS: NPE
INFO:root:MODEL INFERENCE CLASS: NPE
INFO:root:MODEL INFERENCE CLASS: NPE
INFO:root:MODEL INFERENCE CLASS: NPE
INFO:root:MODEL INFERENCE CLASS: NPE
INFO:root:MODEL INFERENCE CLASS: NPE
INFO:root:Training model 1 / 1.
INFO:root:Training model 1 / 1.
INFO:root:Training model 1

 Neural network successfully converged after 120 epochs.

/export/home/vgiusepp/.local/lib/python3.10/site-packages/sbi/utils/torchutils.py:27: UserWarning: GPU was selected as a device for training the neural network. Note that we expect **no** significant speed ups in training for the default architectures we provide. Using the GPU will be effective only for large neural networks with operations that are fast on the GPU, e.g., for a CNN or RNN `embedding_net`.
  warnings.warn(
/export/home/vgiusepp/.local/lib/python3.10/site-packages/sbi/utils/posterior_ensemble.py:142: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self._weights = torch.tensor(weights) / sum(weights)
INFO:root:It took 816.4127707481384 seconds to train models.
[I 2024-05-29 14:28:50,198] Trial 5 finished with value: -4.764560546875 and parameters: {'model': 'nsf', 'hidden_features': 10, 'num_transforms': 10, 'learning_rate':

 Training neural network. Epochs trained: 1321104 epochs.

INFO:root:It took 5950.362004041672 seconds to train models.
[I 2024-05-29 15:54:24,516] Trial 7 finished with value: -1.61337548828125 and parameters: {'model': 'nsf', 'hidden_features': 70, 'num_transforms': 30, 'learning_rate': 1e-05, 'output_dim': 10}. Best is trial 7 with value: -1.61337548828125.
INFO:root:MODEL INFERENCE CLASS: NPE
INFO:root:Training model 1 / 1.


 Training neural network. Epochs trained: 1731682 epochs.

INFO:root:It took 6095.063804149628 seconds to train models.
[I 2024-05-29 15:56:49,190] Trial 4 finished with value: -1.3548538818359375 and parameters: {'model': 'nsf', 'hidden_features': 70, 'num_transforms': 10, 'learning_rate': 1e-05, 'output_dim': 64}. Best is trial 4 with value: -1.3548538818359375.
INFO:root:MODEL INFERENCE CLASS: NPE
INFO:root:Training model 1 / 1.


 Training neural network. Epochs trained: 575